### 1. 生成单条指令，并设置对应的数据

In [1]:
from cimCommand import CMD,CmdData
from cimCommand.singleCmdInfo import *

# 指令所需数据
data=CmdData((1<<24)|7)
# 更改数据位
data.set_bit(3,1)               # 设置数据的第三位为1
data.replace_bit(8,3,22)        # 设置数据从第[8位开始到11)，用22的低三位替换,22中超过3bit会被截断

# 对应的指令，DAC_IN为指令相关信息，如果不设置command_data，就会用DAC_IN里面的默认信息初始化
cmd=CMD(DAC_IN,command_data=data)
print(cmd)

cmd=CMD(DAC_IN,command_data=CmdData((1<<24)|7))
print(cmd)

地址:	2
类型:	RW
位宽:	32 bit
名字:	dac_in
数据:	16778767
字节:	b'\x02\x01\x00\x06\x0f'
描述:	高8bit：0或1，选择dac。低24bit：spi写入DAC寄存器的24bit值

地址:	2
类型:	RW
位宽:	32 bit
名字:	dac_in
数据:	16777223
字节:	b'\x02\x01\x00\x00\x07'
描述:	高8bit：0或1，选择dac。低24bit：spi写入DAC寄存器的24bit值



### 2. 生成指令，并组合成packet数据

In [5]:
from cimCommand import CMD,CmdData,Packet
from cimCommand.singleCmdInfo import *

In [6]:
# 生成packet
pck=Packet()


# 生成对应的指令
cmd_list=[CMD(DAC_IN,command_data=CmdData(7)),CMD(DAC_IN,command_data=CmdData(8)),CMD(DAC_IN,command_data=CmdData(9))]
# 将指令加入packet中
pck.append_single(cmd_list,mode=0)
print(pck)


# 生成对应的指令
cmd_list=[CMD(DAC_IN,command_data=CmdData(7))]
# 将指令加入packet中
pck.append_single(cmd_list,mode=1)
print(pck)


cmd_list=[CMD(DAC_IN,command_data=CmdData(7))]
pck.append_single(cmd_list,mode=2)
print(pck)


cmd_list=[CMD(DAC_IN,command_data=CmdData(7))]
pck.append_single(cmd_list,mode=3)
print(pck)

模式：0,指令：dac_in,指令：dac_in,指令：dac_in
字节码：55 aa 00 02 00 00 00 07 02 00 00 00 08 02 00 00 00 09

模式：0,指令：dac_in,指令：dac_in,指令：dac_in
字节码：55 aa 00 02 00 00 00 07 02 00 00 00 08 02 00 00 00 09
模式：1,指令：dac_in
字节码：55 aa 01 02 00 00 00 07

模式：0,指令：dac_in,指令：dac_in,指令：dac_in
字节码：55 aa 00 02 00 00 00 07 02 00 00 00 08 02 00 00 00 09
模式：1,指令：dac_in
字节码：55 aa 01 02 00 00 00 07
模式：2,指令：dac_in
字节码：55 aa 02 02

模式：0,指令：dac_in,指令：dac_in,指令：dac_in
字节码：55 aa 00 02 00 00 00 07 02 00 00 00 08 02 00 00 00 09
模式：1,指令：dac_in
字节码：55 aa 01 02 00 00 00 07
模式：2,指令：dac_in
字节码：55 aa 02 02
模式：3,指令：dac_in
字节码：55 aa 03



### 3. 将packet的数据发送到下位机

In [1]:
from cimCommand import CMD,CmdData,Packet
from cimCommand.singleCmdInfo import *
from pc import PS
import socket

In [2]:
# 连接下位机
ps = PS("192.168.1.10", 7,debug=True)

# 打包成packet，包括指令
packet=Packet()
cmd_list=[CMD(DAC_IN,command_data=CmdData(7))]
packet.append_cmdlist(cmd_list,mode=1)

try:
    ps.send_packets(packet)                                     # 发送
    packet = ps.receive_packet()                                # 接收
    print(packet)                                               # 打印
except Exception as e:
    print(f"Failed to connect: {e}")
finally:
    ps.close()

Failed to connect: [WinError 10049] 在其上下文中，该请求的地址无效。
None
Connection closed.


### 4. 控制下位机执行相关指令

In [1]:
from modules import ADC,DAC,CHIP
from pc import PS

In [4]:
# 用于和下位机交互通信
ps=PS(1,1,delay=10*1e-3,debug=False)
# 控制ADC
adc=ADC(ps)
# 控制DAC
dac=DAC(ps)
# 芯片，读写操作，device_cfg
chip=CHIP(adc,dac,ps)

In [10]:
ps.set_debug(True)

adc.set_sample_times(32)
dac.set_voltage(5,0,0)
chip.set_device_cfg(pulse_cyc=255)
chip.set_device_cfg(pulse_cyc=1)

------------------------------ 发送指令: ------------------------------ 
模式：1,指令：adc_sample_times
字节码：55 aa 01 18 00 00 00 20

------------------------------ 指令发送完成！------------------------------ 

------------------------------ 发送指令: ------------------------------ 
模式：1,指令：dac_in
字节码：55 aa 01 02 00 08 ff f0
模式：1,指令：fast_command_1
字节码：55 aa 01 01 00 00 00 01

------------------------------ 指令发送完成！------------------------------ 

------------------------------ 发送指令: ------------------------------ 
模式：1,指令：device_cfg
字节码：55 aa 01 07 00 07 ff fc

------------------------------ 指令发送完成！------------------------------ 

------------------------------ 发送指令: ------------------------------ 
模式：1,指令：device_cfg
字节码：55 aa 01 07 00 00 0f fc

------------------------------ 指令发送完成！------------------------------ 

